In [13]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, avg, when, round

In [14]:
# Inicijalizacija Spark sesije
spark = SparkSession.builder.appName("CovidAnalysisSpark").getOrCreate()

#Učitavanje detaset-a 
df_spark = spark.read.csv('../data/Covid_Data_Clean.csv', header=True, inferSchema=True)

#Prikaz prvih 5 redova dataset-a
df_spark.show(5)

#Prikaz tipova podataka
df_spark.printSchema()


+-----+------------+---+-------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+---+-----+---------+-------------------+-----------------+---------+--------------------+------------+
|USMER|MEDICAL_UNIT|SEX|INTUBED|PNEUMONIA|AGE|PREGNANT|DIABETES|COPD|ASTHMA|INMSUPR|HIPERTENSION|OTHER_DISEASE|CARDIOVASCULAR|OBESITY|RENAL_CHRONIC|TOBACCO|ICU|DEATH|AGE_GROUP|COMORBIDITIES_COUNT|COMORBIDITY_GROUP|HIGH_RISK|PNEUMONIA_IN_ELDERLY|HOSPITALIZED|
+-----+------------+---+-------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+---+-----+---------+-------------------+-----------------+---------+--------------------+------------+
|    2|           1|  1|    0.0|      1.0| 65|     0.0|     0.0| 0.0|   0.0|    0.0|         1.0|          0.0|           0.0|    0.0|          0.0|    0.0|0.0|    1|    60-79|                2.0|                2|        1

# ISTRAŽIVAČKO PITANJE 1 - Distribucije ishoda virusa prema starosti i polu

In [15]:
mortality_by_age = df_spark.groupBy("AGE_GROUP") \
    .agg(round(avg("DEATH")*100, 2).alias("death_rate_percent")) \
    .orderBy("AGE_GROUP")
mortality_by_age.show()

+---------+------------------+
|AGE_GROUP|death_rate_percent|
+---------+------------------+
|     0-19|              1.57|
|    20-39|              2.19|
|    40-59|             12.11|
|    60-79|             37.62|
|      80+|             52.98|
+---------+------------------+



Na osnovu dobijene tabele možemo zaključiti da procenat smrtnosti znatno raste sa povećanjem broja godina. Primećuje se da je stopa smrtnosti za uzrast od 0-19 godina 1,57%, dok je za 80+ čak skoro 53%.

In [16]:
#grupisanje smrti po polu
mortality_by_sex = df_spark.groupBy("SEX") \
    .agg(round(avg("DEATH")*100, 2).alias("death_rate_percent"))
mortality_by_sex.show()

+---+------------------+
|SEX|death_rate_percent|
+---+------------------+
|  1|             10.11|
|  0|             16.41|
+---+------------------+



Na osnovu dobijene tabele možemo zaključiti da muškarci (0) imaju malo veću stopu smrtnosti - 16% naprema 10% za žene

# ISTRAŽIVAČKO PITANJE 2 - Faktori hospitalizacije

In [17]:
# Uticaj gojaznosti na hospitalizaciju
hospital_by_obesity = df_spark.groupBy("OBESITY") \
    .agg(round(avg("HOSPITALIZED")*100, 2).alias("hospital_rate_percent"))
hospital_by_obesity.show()

+-------+---------------------+
|OBESITY|hospital_rate_percent|
+-------+---------------------+
|    0.0|                26.37|
|    1.0|                 34.9|
+-------+---------------------+



Možemo zaključiti da gojazni pacijenti imaju veću šansu za hospitalizaciju, skoro 35%, naprema pacijenata koji nisu gojazni -oko 26%

In [18]:
# Procenat hospitalizacije za pacijente sa dijabetesom
hospital_by_diabetes = df_spark.groupBy("DIABETES") \
    .agg(round(avg("HOSPITALIZED")*100, 2).alias("hospital_rate_percent"))
hospital_by_diabetes.show()

+--------+---------------------+
|DIABETES|hospital_rate_percent|
+--------+---------------------+
|     0.0|                22.76|
|     1.0|                55.64|
+--------+---------------------+



Možemo videti da je znatno veća verovatnoća hospitalizacije pacijenata sa dijabetesom - oko 55% 

In [19]:
#prikaz korelacije
corr_diabetes = df_spark.stat.corr("DIABETES", "HOSPITALIZED")
corr_obesity = df_spark.stat.corr("OBESITY", "HOSPITALIZED")

print("Factor       Correlation")
print("------------------------")
print(f"DIABETES     {corr_diabetes}")
print(f"OBESITY      {corr_obesity}")

Factor       Correlation
------------------------
DIABETES     0.26712579658851504
OBESITY      0.07396620413715072


Rezultati pokazuju pozitivnu korelaciju (oko 0.27) između dijabetesa i hospitalizacije, što ukazuje da pacijenti sa dijabetesom imaju povećanu verovatnoću hospitalizacije.
Korelacija između gojaznosti i hospitalizacije je veoma slaba (oko 0.07), što sugeriše da gojaznost nije snažan prediktor hospitalizacije.

# ISTRAŽIVAČKO PITANJE 3 - Komorbiditeti i kritična stanja

In [20]:
#verovatnoća da pacijenti idu na ICU u odnosu na broj hroničnih bolesti
df_spark.groupBy("COMORBIDITY_GROUP").agg(round(avg("ICU")*100,2).alias("icu_prob")).orderBy("COMORBIDITY_GROUP").show()

+-----------------+--------+
|COMORBIDITY_GROUP|icu_prob|
+-----------------+--------+
|                0|    0.29|
|                1|    3.24|
|                2|    5.98|
|               3+|    8.27|
+-----------------+--------+



Vidimo da što je veći broj bolesti, to je veća verovatnoća odlaska na intenzivnu negu

In [21]:
# Filtriranje pacijenata sa 3+ hroničnih bolesti
high_risk_patients = df_spark.filter(col("COMORBIDITY_GROUP") == "3+")

# Grupisanje po intubaciji i brojanje
intubed_count = high_risk_patients.groupBy("INTUBED").count()
intubed_count.show()

+-------+-----+
|INTUBED|count|
+-------+-----+
|    0.0|30896|
|    1.0| 7629|
+-------+-----+



Možemo videti da ima 7629 pacijenata koji imaju 3 ili više bolesti i koji su završili na respiratoru

# ISTRAŽIVAČKO PITANJE 4: Pneumonija, starost i smrtnost

In [22]:
# Filter pacijenata sa pneumonijom i grupisanje po AGE_GROUP
mortality_with_pneumonia = df_spark.filter(col("PNEUMONIA") == 1) \
    .groupBy("AGE_GROUP") \
    .agg(round(avg("DEATH")*100,2).alias("death_rate")) \
    .orderBy("AGE_GROUP")

mortality_with_pneumonia.show()

+---------+----------+
|AGE_GROUP|death_rate|
+---------+----------+
|     0-19|     16.32|
|    20-39|     19.01|
|    40-59|      40.3|
|    60-79|     62.16|
|      80+|     72.44|
+---------+----------+



Što je starija osoba koja ima preumiju, veća je šansa smrtnosti - za osobe sa preko 80 godina čak 72,4% 

In [23]:
# Grupisanje po TOBACCO i računanje prosečne pneumonije
pneumonia_by_smoking = df_spark.groupBy("TOBACCO") \
    .agg(round(avg(col("PNEUMONIA"))*100,2).alias("pneumonia_rate"))

pneumonia_by_smoking.show()

+-------+--------------+
|TOBACCO|pneumonia_rate|
+-------+--------------+
|    0.0|         21.52|
|    1.0|         24.19|
+-------+--------------+



Zabeležena je veća stopa preumonije kod pušača (24,2%), dok je kod nepušača rasprostranjena u 21,52% slučaja